In [1]:
def preprocess_grocery_data(data):
    # Getting the column names
    column_names = data.columns.tolist()

    # Rename existing columns
    rename_mapping = {
        'Day': 'date',
        'Category level 0': 'category_level_0',
        'Category level 1': 'category_level_1',
        'Category level 2': 'category_level_2',
        'Unique SKUs Listed': 'SKUs_listed[Unique]',
        'Unique SKUs Sold': 'SKUs_sold',
        '# of sold SKU items': 'SKUs_items_sold',
        'Sales value before discount (EUR)': 'sale_price_without_discount(euros)',
        'User Discount Value (EUR)': 'user_discounts(euros)',
        'Total COGS for Sold items (net VAT) (EUR)': 'sold_items_total_cogs(euro)',
        'Waste, # of items': 'items_wasted'
    }

    data.rename(columns=rename_mapping, inplace=True)

    # Reordering the columns
    new_column_order = ['date', 'category_level_0', 'category_level_1', 'category_level_2',
                        'SKUs_listed[Unique]', 'SKUs_sold', 'SKUs_items_sold', 'sale_price_without_discount(euros)',
                        'user_discounts(euros)', 'sold_items_total_cogs(euro)', 'items_wasted']

    data = data[new_column_order]

    # Sorting the dataframe
    sort_columns = ['date', 'category_level_0', 'category_level_1', 'category_level_2']
    data = data.sort_values(by=sort_columns)

    # Resetting index
    data.reset_index(drop=True, inplace=True)

    return data


In [2]:
def display_data_info(data):
    # Checking for missing values
    missing_values = data.isnull().sum()
    print("Missing values:")
    print(missing_values)
    print("\nNumber of columns with missing values:", missing_values[missing_values > 0].count())

    # Checking data types
    data_types = data.dtypes
    print("\nData types of columns:")
    print(data_types)



In [3]:
def analyze_zero_values(data):
    # Selecting columns for analysis
    columns_to_check = [
        'SKUs_listed[Unique]',
        'SKUs_sold',
        'SKUs_items_sold',
        'sale_price_without_discount(euros)',
        'user_discounts(euros)',
        'sold_items_total_cogs(euro)'
    ]

    # Filtering rows with all zero values
    zero_values_instances = data[data[columns_to_check].eq(0).all(axis=1)].reset_index(drop=True)

    # Countplots for category levels
    for level in ['category_level_0', 'category_level_1', 'category_level_2']:
        plt.figure(figsize=(12, 6))
        sns.countplot(x=level, data=zero_values_instances, palette='viridis')
        plt.title(f'Zero Values Instances by {level}')
        plt.xlabel(level)
        plt.ylabel('Count')
        plt.xticks(rotation=45, ha='right')
        plt.show()


In [4]:
def data_checks(grocery_sales_inventory_data):
    # Check 1
    check_1_instances = grocery_sales_inventory_data[(grocery_sales_inventory_data['SKUs_listed[Unique]'] == 0) 
                                                     & (grocery_sales_inventory_data['SKUs_sold'] > 0) 
                                                     & (grocery_sales_inventory_data['SKUs_items_sold'] > 0)]
    check_1_count = len(check_1_instances)
    
    # Check 2
    check_2_instances = grocery_sales_inventory_data[(grocery_sales_inventory_data['SKUs_sold'] == 0) 
                                            & (grocery_sales_inventory_data['SKUs_items_sold'] == 0)
                                            & ((grocery_sales_inventory_data['sale_price_without_discount(euros)'] > 0)|
                                              (grocery_sales_inventory_data['user_discounts(euros)'] > 0) |
                                              (grocery_sales_inventory_data['sold_items_total_cogs(euro)'] > 0))]
    check_2_count = len(check_2_instances)
    
    # Check 3
    check_3_instances = grocery_sales_inventory_data[(grocery_sales_inventory_data['SKUs_items_sold'] == 0) 
                                                     & (grocery_sales_inventory_data['items_wasted'] > 0)]
    check_3_count = len(check_3_instances)
    
    # Check 4
    check_4_instances = grocery_sales_inventory_data[(grocery_sales_inventory_data['SKUs_listed[Unique]'] == 0) 
                                                     & (grocery_sales_inventory_data['items_wasted'] > 0)]
    check_4_count = len(check_4_instances)
    
    result = {
        'Check 1': {'count': check_1_count, 'comment': "Instances where SKUs_listed[Unique] is zero with non-zero SKUs_sold and SKUs_items_sold are:"},
        'Check 2': {'count': check_2_count, 'comment': "Instances where sale_price_without_discount, user_discounts, and sold_items_total_cogs are non-zero when SKUs_sold and SKUs_items_sold are zero are:"},
        'Check 3': {'count': check_3_count, 'comment': "Instances where wasted items are non-zero when SKUs_items_sold is zero are:"},
        'Check 4': {'count': check_4_count, 'comment': "Instances where wasted items are non-zero when SKUs_listed[Unique] is zero are:"}
    }
    
    return result



In [6]:
def get_grouped_data_with_zero_sales_info(data):
    # Grouping by the categories and aggregate metrics
    grouped_data = data.groupby(['category_level_0', 'category_level_1', 'category_level_2']).agg({
        'SKUs_listed[Unique]': 'sum',
        'SKUs_sold': 'sum',
        'SKUs_items_sold': 'sum',
        'sale_price_without_discount(euros)': 'sum',
        'user_discounts(euros)': 'sum',
        'sold_items_total_cogs(euro)': 'sum',
        'items_wasted': 'sum'
    }).reset_index()

    # Filtering out instances where sale_price_without_discount(euros) grouped is zero
    filtered_data = grouped_data[
        (grouped_data['sale_price_without_discount(euros)'] == 0) &
        (grouped_data['SKUs_sold'] != 0)
    ]

    # Resetting the index
    filtered_data.reset_index(drop=True, inplace=True)

    # Displaying the size of both dataframes
    print("Total Unique Categories:", grouped_data.shape[0])
    print("Categories with Zero Sales Information:", filtered_data.shape[0])

    return grouped_data, filtered_data



In [ ]:
def applying_feature_engineering(grocery_sales_inventory_data):
    # Assumption 1: Fill missing values in 'sale_price_without_discount(euros)'
    mask = (grocery_sales_inventory_data['sale_price_without_discount(euros)'] == 0)
    grocery_sales_inventory_data['sale_price_without_discount(euros)'] = np.where(mask, 
                                            grocery_sales_inventory_data['sold_items_total_cogs(euro)'] +
                                            grocery_sales_inventory_data['user_discounts(euros)'], 
                                            grocery_sales_inventory_data['sale_price_without_discount(euros)'])

    # Assumption 2: Replace 'SKUs_sold' with 'SKUs_listed' where 'SKUs_listed' is zero
    grocery_sales_inventory_data.loc[(grocery_sales_inventory_data['SKUs_listed[Unique]'] == 0) & 
                                      (grocery_sales_inventory_data['SKUs_sold'] != 0) &
                                      (grocery_sales_inventory_data['SKUs_items_sold'] != 0),
                                     'SKUs_listed[Unique]'] = grocery_sales_inventory_data['SKUs_sold']

    # Introducing new columns: sale_price_with_discount(euros) and net_revenue
    grocery_sales_inventory_data['sale_price_with_discount(euros)'] = \
        grocery_sales_inventory_data['sale_price_without_discount(euros)'] - grocery_sales_inventory_data['user_discounts(euros)']

    grocery_sales_inventory_data['net_revenue'] = \
        grocery_sales_inventory_data['sale_price_with_discount(euros)'] - grocery_sales_inventory_data['sold_items_total_cogs(euro)']

    # Extracting weekday and month from the "date" column
    grocery_sales_inventory_data['weekday'] = pd.to_datetime(grocery_sales_inventory_data['date']).dt.strftime('%a')
    grocery_sales_inventory_data['month'] = pd.to_datetime(grocery_sales_inventory_data['date']).dt.strftime('%b')

    # Reordering the columns
    new_column_order = ['date', 'month', 'weekday', 'category_level_0', 'category_level_1', 'category_level_2',
                        'SKUs_listed[Unique]', 'SKUs_sold', 'SKUs_items_sold', 'sale_price_without_discount(euros)',
                        'user_discounts(euros)', 'sale_price_with_discount(euros)', 'sold_items_total_cogs(euro)',
                        'net_revenue', 'items_wasted']

    grocery_sales_inventory_data = grocery_sales_inventory_data[new_column_order]

    # Sorting the dataframe
    sort_columns = ['date', 'category_level_0', 'category_level_1', 'category_level_2', 'net_revenue']
    grocery_sales_inventory_data = grocery_sales_inventory_data.sort_values(by=sort_columns)

    # Resetting index
    grocery_sales_inventory_data.reset_index(drop=True, inplace=True)

    # List of columns to check for zero values
    columns_to_check = ['SKUs_listed[Unique]', 'SKUs_sold', 'SKUs_items_sold', 
                         'sale_price_without_discount(euros)', 'user_discounts(euros)', 
                         'sale_price_with_discount(euros)', 'sold_items_total_cogs(euro)', 
                         'net_revenue', 'items_wasted']

    # Drop rows where specified columns have zero values
    grocery_sales_inventory_data_final = grocery_sales_inventory_data[grocery_sales_inventory_data[columns_to_check].ne(0).any(axis=1)]

    # Reset the index
    grocery_sales_inventory_data_final.reset_index(drop=True, inplace=True)

    return grocery_sales_inventory_data_final


In [10]:

import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns

def convert_seaborn_palette(seaborn_palette):
    return [f'rgb({int(r * 255)},{int(g * 255)},{int(b * 255)})' for r, g, b in seaborn_palette]

def plot_overall_monthly_aggregated_metrics(df):
    # Define a specific sequence of months
    month_order = ['Apr', 'May', 'Jun']
    
    columns_to_aggregate = ['sale_price_without_discount(euros)', 'user_discounts(euros)', 'net_revenue', 'SKUs_items_sold','items_wasted']

    # Group by month and aggregate the specified columns
    agg_data_monthly = df.groupby('month')[columns_to_aggregate].sum().reset_index()
    
    # Sort the aggregated data based on the custom month order
    agg_data_monthly['month'] = pd.Categorical(agg_data_monthly['month'], categories=month_order, ordered=True)
    agg_data_monthly = agg_data_monthly.sort_values('month')

    # Define a color palette with higher tones of green
    green_palette = sns.color_palette("dark:#38A169", len(columns_to_aggregate))
    plotly_palette = convert_seaborn_palette(green_palette)

    # Create a line plot for each metric
    fig = go.Figure()

    for i, column in enumerate(columns_to_aggregate):
        fig.add_trace(go.Scatter(x=agg_data_monthly['month'], y=agg_data_monthly[column],
                                 mode='lines+markers', name=column, line=dict(color=plotly_palette[i])))

    fig.update_layout(title='Overall Monthly Aggregated Metrics',
                      xaxis_title='Month',
                      yaxis_title='Values')

    fig.show()


In [11]:

def convert_seaborn_palette(seaborn_palette):
    return [f'rgb({int(r * 255)},{int(g * 255)},{int(b * 255)})' for r, g, b in seaborn_palette]

def plot_category_level_0_monthly_aggregated_metrics(df):
    # Define a specific sequence of months
    month_order = ['Apr', 'May', 'Jun']
    
    columns_to_aggregate = ['sale_price_without_discount(euros)', 'user_discounts(euros)', 'net_revenue', 'SKUs_items_sold', 'items_wasted']

    # Group by category_level_0, month, and aggregate the specified columns
    agg_data_category_month = df.groupby(['category_level_0', 'month'])[columns_to_aggregate].sum().reset_index()
    
    # Sort the aggregated data based on the custom month order
    agg_data_category_month['month'] = pd.Categorical(agg_data_category_month['month'], categories=month_order, ordered=True)
    agg_data_category_month = agg_data_category_month.sort_values(['category_level_0', 'month'])

    # Define a color palette with higher tones of green
    green_palette = sns.color_palette("dark:#38A169", len(columns_to_aggregate))
    plotly_palette = convert_seaborn_palette(green_palette)

    for column in columns_to_aggregate:
        # Create line plots for each category_level_0
        fig = go.Figure()
        for i, category in enumerate(agg_data_category_month['category_level_0'].unique()):
            subset = agg_data_category_month[agg_data_category_month['category_level_0'] == category]
            fig.add_trace(go.Scatter(x=subset['month'], y=subset[column],
                                     mode='lines+markers', name=f'{category} - {column}', line=dict(color=plotly_palette[i])))
        fig.update_layout(title=f'Category Level 0 Monthly Aggregated Metrics - {column}',
                          xaxis_title='Month',
                          yaxis_title='Values')
        fig.show()

        # Print calculated insights for each category_level_0
        for category in agg_data_category_month['category_level_0'].unique():
            subset_apr = agg_data_category_month[(agg_data_category_month['category_level_0'] == category) & (agg_data_category_month['month'] == 'Apr')]
            subset_may = agg_data_category_month[(agg_data_category_month['category_level_0'] == category) & (agg_data_category_month['month'] == 'May')]
            subset_jun = agg_data_category_month[(agg_data_category_month['category_level_0'] == category) & (agg_data_category_month['month'] == 'Jun')]
            
            if not subset_apr.empty and not subset_may.empty and not subset_jun.empty:
                print(f"** Insights for Category: {category} - {column} **")
                diff_apr_may = subset_may[column].values[0] - subset_apr[column].values[0]
                diff_may_jun = subset_jun[column].values[0] - subset_may[column].values[0]

                print(f"Values increased from Apr to May by {abs(diff_apr_may):,.2f}." if diff_apr_may > 0 else
                      f"Values decreased from Apr to May by {abs(diff_apr_may):,.2f}." if diff_apr_may < 0 else
                      "Values remained constant from Apr to May.")

                print(f"Values increased from May to Jun by {abs(diff_may_jun):,.2f}." if diff_may_jun > 0 else
                      f"Values decreased from May to Jun by {abs(diff_may_jun):,.2f}." if diff_may_jun < 0 else
                      "Values remained constant from May to Jun.")
                print("\n")



In [13]:

def aggregate_and_display_category_level(df):
    
    category_level = ['category_level_0', 'category_level_1', 'category_level_2']
    
    columns_to_aggregate = ['sale_price_without_discount(euros)', 'user_discounts(euros)', 'net_revenue','SKUs_listed[Unique]' ,'SKUs_sold','SKUs_items_sold', 'items_wasted']
    
    # Group by category_level and aggregate the specified columns
    agg_data_category_level = df.groupby(category_level)[columns_to_aggregate].sum().reset_index()

    # Sort the aggregated data by 'net_revenue' in descending order
    agg_data_category_level = agg_data_category_level.sort_values(by='net_revenue', ascending=False)

    # Reindex the DataFrame
    agg_data_category_level = agg_data_category_level.reset_index(drop=True)
    
    return agg_data_category_level




In [14]:
def plot_wastage_by_category(df, category_level, threshold=None):
    # Group by category_level and calculate total wastage
    wastage_by_category = df.groupby(category_level)['items_wasted'].sum().sort_values(ascending=False)

    # Calculate percentage of total wastage
    total_wastage = wastage_by_category.sum()
    wastage_percentage = (wastage_by_category / total_wastage) * 100

    # Apply threshold if specified
    if threshold is not None:
        filtered_wastage = wastage_percentage[wastage_by_category > threshold]
    else:
        filtered_wastage = wastage_percentage

    # Reverse the color scale for lighter tones to represent lower percentages
    colorscale = px.colors.sequential.Greens[::-1]

    # Plot the bar chart
    fig = px.bar(filtered_wastage, x=filtered_wastage.values, y=filtered_wastage.index, orientation='h',
                 title=f'Categories with High Wastage [{category_level}] (Wastage > {threshold}%)' if threshold is not None else f'Categories with Wastage Percentage [{category_level}]',
                 labels={'x': 'Percentage of Total Wastage', 'y': f'{category_level}'},
                 color=filtered_wastage.values,
                 color_continuous_scale=colorscale)
    
    # Show the plot
    fig.show()


In [15]:
def plot_monthly_wastage_trends_by_category(df, category_level):
    # Convert the 'date' column to datetime
    df['date'] = pd.to_datetime(df['date'])
    
    # Create a copy of the DataFrame
    monthly_wastage_trends = df.copy()
    
    # Extract month and year from the 'date' column
    monthly_wastage_trends['month'] = monthly_wastage_trends['date'].dt.to_period("M")
    
    # Group by category_level and month, and calculate total wastage
    monthly_wastage_trends = monthly_wastage_trends.groupby([category_level, 'month'])['items_wasted'].sum().reset_index()

    # Convert the 'month' column to string
    monthly_wastage_trends['month'] = monthly_wastage_trends['month'].astype(str)

    # Create a bar chart with trend lines
    fig = px.bar(monthly_wastage_trends, x='month', y='items_wasted', color=category_level,
                 title=f'Monthly Item Wastage Trends by {category_level}',
                 labels={'items_wasted': 'Total Wasted Items'},
                 hover_name=category_level, text='items_wasted',
                 color_discrete_sequence=px.colors.sequential.Greens_r)  # Use reversed Greens color scale

    # Add trend lines
    fig.update_traces(
        marker=dict(color=px.colors.sequential.Greens_r[-1]),
        line=dict(color=px.colors.sequential.Greens_r[-1], width=3),
        selector=dict(mode='lines')
    )
    
    fig.update_layout(showlegend=True, xaxis_title='Month', yaxis_title='Total Wasted Items', legend_title=category_level)

    fig.show()

In [16]:

def plot_daily_wastage_trends_by_category_level_0(df):
    # Convert the 'date' column to datetime
    df['date'] = pd.to_datetime(df['date'])
    
    # Group by category_level_0 and date, and calculate total wastage
    daily_wastage_trends = df.groupby(['category_level_0', 'date'])['items_wasted'].sum().reset_index()

    # Create a line chart with markers using dark green tones
    fig = px.line(daily_wastage_trends, x='date', y='items_wasted', color='category_level_0', markers=True,
                  title='Daily Item Wastage Trends by Category Level 0', labels={'items_wasted': 'Total Wasted Items'},
                  color_discrete_sequence=px.colors.sequential.Greens_r)  # Use reversed Greens color scale

    fig.update_traces(marker=dict(size=10, line=dict(color='white', width=2)),
                      line=dict(width=3))

    fig.update_layout(xaxis_title='Date', yaxis_title='Total Wasted Items', legend_title='Category Level 0',
                      legend=dict(title=dict(text='Category Level 0'), orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1),
                      plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)')

    fig.show()


In [ ]:
def plot_daily_wastage_trends_cat1_grocery_food(df):
    # Filter data for Grocery Food at Category Level 0
    filtered_data_cat2 = df[df['category_level_0'].isin(['Grocery Food'])]

    # Group by Category Level 0, Category Level 1, and date, and calculate total wastage
    daily_wastage_trends_cat2 = filtered_data_cat2.groupby(['category_level_0', 'category_level_1', 'date'])['items_wasted'].sum().reset_index()

    # Create a line chart with markers using darker green tones
    fig_cat2 = px.line(daily_wastage_trends_cat2, x='date', y='items_wasted', color='category_level_1', markers=True,
                       title='Daily Item Wastage Trends by Category Level 1 within Grocery Food',
                       labels={'items_wasted': 'Total Wasted Items'},
                       color_discrete_sequence=px.colors.sequential.Greens_r)  # Use reversed Greens color scale

    fig_cat2.update_traces(marker=dict(size=10, line=dict(width=2)))

    fig_cat2.update_layout(xaxis_title='Date', yaxis_title='Total Wasted Items', legend_title='Category Level 1',
                          legend=dict(title=dict(text='Category Level 1'), orientation='v', yanchor='top', xanchor='right', x=1.02, y=1),
                          plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)')

    fig_cat2.show()




In [17]:
def plot_daily_wastage_trends_fresh_food(df):
    # Filter data for Fresh Food at Category Level 0
    filtered_data_cat2 = df[df['category_level_0'].isin(['Fresh Food'])]

    # Group by Category Level 0, Category Level 1, and date, and calculate total wastage
    daily_wastage_trends_cat2 = filtered_data_cat2.groupby(['category_level_0', 'category_level_1', 'date'])['items_wasted'].sum().reset_index()

    # Create a line chart with markers using darker green tones
    fig_cat2 = px.line(daily_wastage_trends_cat2, x='date', y='items_wasted', color='category_level_1', markers=True,
                       title='Daily Item Wastage Trends by Category Level 1 within Fresh Food',
                       labels={'items_wasted': 'Total Wasted Items'},
                       color_discrete_sequence=px.colors.sequential.Greens_r)  # Use reversed Greens color scale

    fig_cat2.update_traces(marker=dict(size=10, line=dict(width=2)))

    fig_cat2.update_layout(xaxis_title='Date', yaxis_title='Total Wasted Items', legend_title='Category Level 1',
                          legend=dict(title=dict(text='Category Level 1'), orientation='v', yanchor='top', xanchor='right', x=1.02, y=1),
                          plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)')

    fig_cat2.show()

In [ ]:
def plot_sales_trends(df):
    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values(by='date')

    # Create a categorical column for month, weekday, and date with the correct sequence
    month_order = ['Apr', 'May', 'Jun']
    weekday_order = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']

    df['month'] = pd.Categorical(df['month'], categories=month_order, ordered=True)
    df['weekday'] = pd.Categorical(df['weekday'], categories=weekday_order, ordered=True)
    df['date'] = pd.to_datetime(df['date']).dt.date

    # Group by month, weekday, and date and sum up the values
    df_monthly = df.groupby('month').sum().reset_index()
    df_weekly = df.groupby('weekday').sum().reset_index()
    df_daily = df.groupby('date').sum().reset_index()

    # Create a trend graph using plotly express
    fig_monthly = px.line(df_monthly, x='month', y='sale_price_without_discount(euros)', title='Monthly Trend',
                          labels={'sale_price_without_discount(euros)': 'Total Sales', 'month': 'Month'},
                          line_shape='linear', color_discrete_sequence=px.colors.sequential.Greens_r)
    fig_weekly = px.line(df_weekly, x='weekday', y='sale_price_without_discount(euros)', title='Weekday Trend',
                         labels={'sale_price_without_discount(euros)': 'Total Sales', 'weekday': 'Weekday'},
                         line_shape='linear', color_discrete_sequence=px.colors.sequential.Greens_r)
    fig_daily = px.line(df_daily, x='date', y='sale_price_without_discount(euros)', title='Date-wise Trend',
                        labels={'sale_price_without_discount(euros)': 'Total Sales', 'date': 'Date'},
                        line_shape='linear', color_discrete_sequence=px.colors.sequential.Greens_r)

    # Update y-axis formatting for better readability
    fig_monthly.update_layout(yaxis=dict(tickformat=',.2f'))
    fig_weekly.update_layout(yaxis=dict(tickformat=',.2f'))
    fig_daily.update_layout(yaxis=dict(tickformat=',.2f'))

    # Show hover information
    fig_monthly.update_traces(mode='lines+markers', hovertemplate='%{x}<br>%{y:,.2f}')
    fig_weekly.update_traces(mode='lines+markers', hovertemplate='%{x}<br>%{y:,.2f}')
    fig_daily.update_traces(mode='lines+markers', hovertemplate='%{x}<br>%{y:,.2f}')

    # Show the interactive plots
    fig_monthly.show()
    fig_weekly.show()
    fig_daily.show()


In [19]:
def plot_sku_listed_trends(df):
    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values(by='date')

    # Create a categorical column for month, weekday, and date with the correct sequence
    month_order = ['Apr', 'May', 'Jun']
    weekday_order = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']

    df['month'] = pd.Categorical(df['month'], categories=month_order, ordered=True)
    df['weekday'] = pd.Categorical(df['weekday'], categories=weekday_order, ordered=True)
    df['date'] = pd.to_datetime(df['date']).dt.date

    # Group by month, weekday, and date and get unique SKUs listed
    df_monthly_sku = df.groupby('month')['SKUs_listed[Unique]'].nunique().reset_index()
    df_weekly_sku = df.groupby('weekday')['SKUs_listed[Unique]'].nunique().reset_index()
    df_daily_sku = df.groupby('date')['SKUs_listed[Unique]'].nunique().reset_index()

    # Create a trend graph using plotly express
    fig_monthly = px.line(df_monthly_sku, x='month', y='SKUs_listed[Unique]', title='Monthly SKU Listed Trends',
                          labels={'SKUs_listed[Unique]': 'Unique SKUs Listed', 'month': 'Month'},
                          line_shape='linear', color_discrete_sequence=px.colors.sequential.Greens_r)
    fig_weekly = px.line(df_weekly_sku, x='weekday', y='SKUs_listed[Unique]', title='Weekly SKU Listed Trends',
                         labels={'SKUs_listed[Unique]': 'Unique SKUs Listed', 'weekday': 'Weekday'},
                         line_shape='linear', color_discrete_sequence=px.colors.sequential.Greens_r)
    fig_daily = px.line(df_daily_sku, x='date', y='SKUs_listed[Unique]', title='Daily SKU Listed Trends',
                        labels={'SKUs_listed[Unique]': 'Unique SKUs Listed', 'date': 'Date'},
                        line_shape='linear', color_discrete_sequence=px.colors.sequential.Greens_r)

    # Update y-axis formatting for better readability
    fig_monthly.update_layout(yaxis=dict(tickformat=',.0f'))
    fig_weekly.update_layout(yaxis=dict(tickformat=',.0f'))
    fig_daily.update_layout(yaxis=dict(tickformat=',.0f'))

    # Show hover information
    fig_monthly.update_traces(mode='lines+markers', hovertemplate='%{x}<br>%{y:,.0f}')
    fig_weekly.update_traces(mode='lines+markers', hovertemplate='%{x}<br>%{y:,.0f}')
    fig_daily.update_traces(mode='lines+markers', hovertemplate='%{x}<br>%{y:,.0f}')

    # Show the interactive plots
    fig_monthly.show()
    fig_weekly.show()
    fig_daily.show()

In [ ]:

def analyze_category_level_1_trends(df):
    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values(by='date')

    month_order = ['Apr', 'May', 'Jun']
    weekday_order = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']

    df['month'] = pd.Categorical(df['month'], categories=month_order, ordered=True)
    df['weekday'] = pd.Categorical(df['weekday'], categories=weekday_order, ordered=True)
    df['date'] = pd.to_datetime(df['date']).dt.date

    # Grouping by month, weekday, and date and sum up the values
    df_monthly = df.groupby(['month', 'category_level_1']).sum().reset_index()
    df_weekly = df.groupby(['weekday', 'category_level_1']).sum().reset_index()
    df_daily = df.groupby(['date', 'category_level_1']).sum().reset_index()

    # Reshape data to have months as columns and categories as rows
    pivot_monthly = df_monthly.pivot(index='category_level_1', columns='month', values='sale_price_without_discount(euros)').fillna(0)

    # Calculate the difference between June and April sales
    sales_diff = pivot_monthly['Jun'] - pivot_monthly['Apr']

    # Identify categories with a decrease or increase in sales
    decrease_categories = sales_diff[sales_diff < 0].index.tolist()
    increase_categories = sales_diff[sales_diff > 0].index.tolist()

    # Print the results
    print("\nMonthly Trends at category level 1:")
    print("Decrease:", decrease_categories)
    print("Increase:", increase_categories)



In [21]:
def plot_high_margin_skus_category_level_1(df):
    # Assuming your dataframe is named df
    df['profit_margin'] = (df['sale_price_without_discount(euros)'] - df['sold_items_total_cogs(euro)'])  / df['sale_price_without_discount(euros)']

    # Use .loc to avoid SettingWithCopyWarning
    high_margin_skus = df.loc[df['profit_margin'] > 0.1].copy()

    # Create an interactive boxplot with plotly.express
    fig = px.box(high_margin_skus,
                 x='category_level_1',
                 y='profit_margin',
                 color='category_level_1',
                 title='SKUs with Higher Margins',
                 labels={'profit_margin': 'Profit Margin', 'category_level_1': 'Category Level 1'},
                 template='plotly_dark')

    # Customize the box colors with dark green shades
    dark_green_colorscale = px.colors.sequential.Greens[::-1]  # Reverse the Greens colorscale
    color_indices = [i * (len(dark_green_colorscale) - 1) // len(high_margin_skus['category_level_1'].unique()) for i in range(len(high_margin_skus['category_level_1'].unique()))]

    # Update boxpoints and pointpos to customize the marker position and color
    for i, index in enumerate(color_indices):
        # Ensure that pointpos falls within the acceptable range [-2, 2]
        pointpos = max(-2, min(2, -index))
        fig.data[i].update(boxpoints='all', pointpos=pointpos, jitter=0.3, marker=dict(color=dark_green_colorscale[index]))

    # Set light green background
    fig.update_layout(plot_bgcolor=px.colors.sequential.Greens[0])

    # Show the interactive plot
    fig.show()



In [22]:
def plot_conversion_rate_by_category(df):
    """
    Plots the conversion rate by category using Plotly Express box plot.

    Parameters:
    - df (pd.DataFrame): The input DataFrame containing the necessary columns.

    Returns:
    - fig (plotly.graph_objects.Figure): The Plotly figure object.
    """
    # Calculate conversion rate
    df['conversion_rate'] = df['SKUs_sold'] / df['SKUs_listed[Unique]']

    # Filter outliers (conversion_rate between 0 and 1)
    df = df[(df['conversion_rate'] >= 0) & (df['conversion_rate'] <= 1)]

    # Plotly Express for interactive visualization
    fig = px.box(df, x='conversion_rate', y='category_level_1', color='category_level_1',
                 title='Conversion Rate by Category', labels={'conversion_rate': 'Conversion Rate'})
    
    # Set green color tones for background and boxplot
    fig.update_layout(plot_bgcolor=px.colors.sequential.Greens[0])

    # Customize axes titles
    fig.update_layout(xaxis=dict(title='Conversion Rate'), yaxis=dict(title='Category Level 1'))

    return fig



In [23]:
import pandas as pd

def categorize_categories(df, high_net_margin_threshold=0.25, high_conversion_rate_threshold=0.40,
                           low_net_margin_threshold=0.24, low_conversion_rate_threshold=0.39):
    """
    Categorizes categories at category_level_1 based on net margin and conversion rate thresholds.

    Parameters:
    - df (pd.DataFrame): The input DataFrame containing the necessary columns.
    - high_net_margin_threshold (float): Threshold for high net margin.
    - high_conversion_rate_threshold (float): Threshold for high conversion rate.
    - low_net_margin_threshold (float): Threshold for low net margin.
    - low_conversion_rate_threshold (float): Threshold for low conversion rate.

    Returns:
    - result_dict (dict): A dictionary containing categories categorized based on thresholds.
    """
    # Calculate net margin and conversion rate
    df['net_margin'] = df['sold_items_total_cogs(euro)'] / df['sale_price_without_discount(euros)']
    df['conversion_rate'] = df['SKUs_sold'] / df['SKUs_listed[Unique]']

    # Group by category_level_0 and identify categories with high net margin and high conversion rate
    grouped_data = df.groupby('category_level_1').agg({
        'net_margin': 'mean',
        'conversion_rate': 'mean'
    })

    # Reset the index to avoid the KeyError
    grouped_data.reset_index(inplace=True)

    # Categorize categories based on thresholds
    high_net_high_conversion = grouped_data[(grouped_data['net_margin'] > high_net_margin_threshold) &
                                            (grouped_data['conversion_rate'] > high_conversion_rate_threshold)]

    high_net_low_conversion = grouped_data[(grouped_data['net_margin'] > high_net_margin_threshold) &
                                           (grouped_data['conversion_rate'] < low_conversion_rate_threshold)]

    low_net_high_conversion = grouped_data[(grouped_data['net_margin'] < low_net_margin_threshold) &
                                           (grouped_data['conversion_rate'] > high_conversion_rate_threshold)]

    low_net_low_conversion = grouped_data[(grouped_data['net_margin'] < low_net_margin_threshold) &
                                          (grouped_data['conversion_rate'] < low_conversion_rate_threshold)]

    # Prepare the result dictionary
    result_dict = {
        'high_net_high_conversion': high_net_high_conversion['category_level_1'].unique(),
        'high_net_low_conversion': high_net_low_conversion['category_level_1'].unique(),
        'low_net_high_conversion': low_net_high_conversion['category_level_1'].unique(),
        'low_net_low_conversion': low_net_low_conversion['category_level_1'].unique()
    }

    return result_dict


In [24]:
def plot_sales_vs_discounts(df):
    """
    Plots a scatter plot of sales vs. discounts with a regression line using Plotly Express.

    Parameters:
    - df (pd.DataFrame): The input DataFrame containing the necessary columns.

    Returns:
    - None: Displays the interactive plot.
    """
    # Scatter plot with regression line
    fig = px.scatter(df,
                     x='SKUs_sold',
                     y='user_discounts(euros)',
                     color='category_level_2',
                     size='sale_price_without_discount(euros)',
                     hover_name='category_level_2',
                     labels={'SKUs_sold': 'Number of SKUs Sold', 'user_discounts(euros)': 'User Discounts (EUR)'},
                     title='Sales vs. Discounts Scatter Plot with Regression Line',
                     template='plotly_dark',
                     trendline='ols')

    # Update marker style
    fig.update_traces(marker=dict(line=dict(width=0.5, color='DarkSlateGray')),
                      selector=dict(mode='markers'))

    # Add gridlines for better readability
    fig.update_layout(xaxis=dict(showgrid=True, gridwidth=1, gridcolor='LightGray'),
                      yaxis=dict(showgrid=True, gridwidth=1, gridcolor='LightGray'))

    # Update marker color to green shades
    green_colorscale = px.colors.sequential.Greens
    fig.update_traces(marker=dict(color=df['user_discounts(euros)'],
                                  colorscale=green_colorscale),
                      selector=dict(mode='markers'))

    # Show the interactive plot
    fig.show()


In [ ]:
def category_bifurcation_on_discount_sensitivity(df):
    """
    Plots a scatter plot of sales vs. discounts with a regression line for each category,
    categorizing categories based on the slope of the regression line.

    Parameters:
    - df (pd.DataFrame): The input DataFrame containing the necessary columns.

    Returns:
    - None: Displays the interactive plot.
    """
    # Scatter plot with regression line
    fig = px.scatter(df,
                     x='SKUs_sold',
                     y='user_discounts(euros)',
                     color='category_level_2',
                     size='sale_price_without_discount(euros)',
                     hover_name='category_level_2',
                     labels={'SKUs_sold': 'Number of SKUs Sold', 'user_discounts(euros)': 'User Discounts (EUR)'},
                     title='Sales vs. Discounts Scatter Plot with Regression Line',
                     template='plotly_dark')

    # Calculate the slope of the regression line for each category
    slopes = []

    for category in df['category_level_2'].unique():
        category_data = df[df['category_level_2'] == category]
        results = sm.OLS(category_data['user_discounts(euros)'],
                         sm.add_constant(category_data['SKUs_sold'])).fit()
        slope = results.params['SKUs_sold']
        slopes.append({'category_level_2': category, 'slope': slope})

    # Convert slopes to a DataFrame
    slopes_df = pd.DataFrame(slopes)

    # Define a threshold to determine if discounting matters
    slope_threshold = 0.5

    # Categorize based on the slope
    categories_matter = slopes_df[slopes_df['slope'] >= slope_threshold]['category_level_2'].values
    categories_do_not_matter = slopes_df[slopes_df['slope'] < slope_threshold]['category_level_2'].values

    # Print the results
    print("Categories where discounting matters:")
    print(categories_matter)

    print("\nCategories where discounting doesn't matter:")
    print(categories_do_not_matter)

    # Update marker style
    #fig.update_traces(marker=dict(line=dict(width=0.5, color='DarkSlateGray')),
    #                  selector=dict(mode='markers'))

    # Add gridlines for better readability
    #fig.update_layout(xaxis=dict(showgrid=True, gridwidth=1, gridcolor='LightGray'),
    #                  yaxis=dict(showgrid=True, gridwidth=1, gridcolor='LightGray'))

    # Show the interactive plot
    #fig.show()



In [25]:
def plot_waste_trends(df):
    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values(by='date')

    # Create a categorical column for month, weekday, and date with the correct sequence
    month_order = ['Apr', 'May', 'Jun']
    weekday_order = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']

    df['month'] = pd.Categorical(df['month'], categories=month_order, ordered=True)
    df['weekday'] = pd.Categorical(df['weekday'], categories=weekday_order, ordered=True)
    df['date'] = pd.to_datetime(df['date']).dt.date

    # Group by month, weekday, and date and sum up the values
    df_monthly = df.groupby('month').sum().reset_index()
    df_weekly = df.groupby('weekday').sum().reset_index()
    df_daily = df.groupby('date').sum().reset_index()

    # Create a trend graph using plotly express
    fig_monthly = px.line(df_monthly, x='month', y='items_wasted', title='Monthly Trend - Items Wasted',
                          labels={'items_wasted': 'Total Items Wasted', 'month': 'Month'},
                          line_shape='linear', color_discrete_sequence=px.colors.sequential.Greens_r)
    fig_weekly = px.line(df_weekly, x='weekday', y='items_wasted', title='Weekday Trend - Items Wasted',
                         labels={'items_wasted': 'Total Items Wasted', 'weekday': 'Weekday'},
                         line_shape='linear', color_discrete_sequence=px.colors.sequential.Greens_r)
    fig_daily = px.line(df_daily, x='date', y='items_wasted', title='Date-wise Trend - Items Wasted',
                        labels={'items_wasted': 'Total Items Wasted', 'date': 'Date'},
                        line_shape='linear', color_discrete_sequence=px.colors.sequential.Greens_r)

    # Update y-axis formatting for better readability
    fig_monthly.update_layout(yaxis=dict(tickformat=',.2f'))
    fig_weekly.update_layout(yaxis=dict(tickformat=',.2f'))
    fig_daily.update_layout(yaxis=dict(tickformat=',.2f'))

    # Show hover information
    fig_monthly.update_traces(mode='lines+markers', hovertemplate='%{x}<br>%{y:,.2f}')
    fig_weekly.update_traces(mode='lines+markers', hovertemplate='%{x}<br>%{y:,.2f}')
    fig_daily.update_traces(mode='lines+markers', hovertemplate='%{x}<br>%{y:,.2f}')

    # Show the interactive plots
    fig_monthly.show()
    fig_weekly.show()
    fig_daily.show()